# Gráfico Função de Michaelis (Km e Vmáx ajustável)

In [3]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

#Função de velocidade
def michaelis (conc_subst, km, vmax):
    v_reaction = (vmax*conc_subst)/(km+conc_subst)
    return v_reaction

#Cria Figura (provisória)
fig = go.Figure()

#Cria app
app = Dash(__name__)

#Define visual do app
app.layout = html.Div([
    #Cria Slider Km
    html.Label("Km:", style={'color': 'white'}),
    dcc.Slider(
        id='km-slider',
        min=0.1,
        max=2.0,
        step=0.1,
        value=0.4
    ),

    html.Br(),

    #Cria Slider Vmáx
    html.Label("Vmax:", style={'color': 'white'}),
    dcc.Slider(
        id='vmax-slider',
        min=10,
        max=50,
        step=1,
        value=32
    ),

    #Insere Gráfico
    dcc.Graph(id='Velocidade(concentração)', figure=fig)
])

#Define Inputs e Outputs
@app.callback(
    Output('Velocidade(concentração)', 'figure'),
    Input('km-slider', 'value'),
    Input('vmax-slider', 'value')
)

#Atualiza o gráfico
def mudar_imagem(km, vmax):

    #Concentração x Velocidade
    s = np.linspace(0, 20, 500)
    v = michaelis(s, km, vmax)

    #Cria figura
    fig = go.Figure()

    #Adiciona pontos/linhas
    fig.add_trace(go.Scatter(
        x=s,
        y=v,
        mode='lines',
    ))

    #Define visual
    fig.update_layout(
        title=f"Função de Michaelis-Meten - km = {km}, Vmáx = {vmax}",
        xaxis=dict(range=[0, 20]),
        yaxis=dict(range=[0, 50]),
        xaxis_title="Concentração do Substrato (mol/L)",
        yaxis_title="Velocidade da reação (mol/L.s)"
    )

    return fig

#Roda app
if __name__ == '__main__':
    app.run(debug=True, port=8030)   

# Grafico em função de temperatura e pH

In [4]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import math

#Função de velocidade
def v0(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):
    term1 = 1 / (1 + 10**(pKa - pH) + 10**(pH - pKb))
    exp1 = math.exp((-Ea*1000 / 8.314) * (1/(T+273) - 1/(T_ref+273)))
    exp2 = math.exp((deltaH*1000 / 8.314) * (1/(T+273) - 1/(T_ref+273)))
    exp3 = math.exp(-((T - T_opt) / sigma)**2)
    
    
    numerador = Vmax_ref * exp1 * S
    denominador = Km_ref * exp2 + S
    
    return term1 * exp3 * (numerador / denominador)

#Cria Figura (provisória)
fig = go.Figure()

#Cria app
app = Dash(__name__)

#Define visual do app
app.layout = html.Div([
    #Cria Input pKa
    html.Label("Constante de dissociação do grupo ácido (pKa)", style={'color': 'white'}),
    dcc.Input(
        id='input_pka',
        type='number',
        placeholder="Insira o valor de pKa",
        value='6.5'),

    html.Br(),

    #Cria Input pKb
    html.Label("Constante de dissociação do grupo básico (pKb)", style={'color': 'white'}),
    dcc.Input(
        id='input_pkb',
        type='number',
        placeholder="Insira o valor de pKb",
        value='8'),

    html.Br(),

    #Cria Input T_ref
    html.Label("Temperatura referêncial (T_ref)", style={'color': 'white'}),
    dcc.Input(
        id='input_t_ref',
        type='number',
        placeholder="Insira o valor de T_ref",
        value='25'),

    html.Br(),

    #Cira Input Vmax_ref
    html.Label("Velocidade máxima na temperatura referencial (Vmax_ref)", style={'color': 'white'}),
    dcc.Input(
        id='input_vmax_ref',
        type='number',
        placeholder="Insira o valor de Vmax_ref",
        value='32'),

    html.Br(),

    #Cria Input Km_ref
    html.Label("Constante de Michaelis na temperatura referencial (Km_ref)", style={'color': 'white'}),
    dcc.Input(
        id='input_km_ref',
        type='number',
        placeholder="Insira o valor de Km_ref",
        value='0.4'),

    html.Br(),

    #Cria Input Ea
    html.Label("Energia de ativação (Ea)", style={'color': 'white'}),
    dcc.Input(
        id='input_ea',
        type='number',
        placeholder="Insira o valor de Ea",
        value='50'),

    html.Br(),

    #Cria Input ΔH
    html.Label("Variação de Entalpia (ΔH)", style={'color': 'white'}),
    dcc.Input(
        id='input_deltah',
        type='number',
        placeholder="Insira o valor de ΔH",
        value='-20'),

    html.Br(),

    #Cira Input T_opt
    html.Label("Temperatura Ótima de funcionamento (T_opt)", style={'color': 'white'}),
    dcc.Input(
        id='input_t_opt',
        type='number',
        placeholder="Insira o valor de T_opt",
        value='20'),
    
    html.Br(),

    #Cria Input σ
    html.Label("Sensibilidade térmica (σ))", style={'color': 'white'}),
    dcc.Input(
        id='input_sigma',
        type='number',
        placeholder="Insira o valor de σ",
        value='45'),
    
    html.Br(),

    #Cria Slider Temperatura
    html.Label("Temperatura:", style={'color': 'white'}),
    dcc.Slider(
        id='t-slider',
        min=20,
        max=70,
        step=1,
        value=25
    ),

    html.Br(),

    #Cria Slider pH
    html.Label("pH:", style={'color': 'white'}),
    dcc.Slider(
        id='ph-slider',
        min=5.5,
        max=9.5,
        step=0.1,
        value=7
    ),

    #Insere Gráfico
    dcc.Graph(id='Velocidade(concentração)', figure=fig)
])

#Define Inputs e Outputs
@app.callback(
    Output('Velocidade(concentração)', 'figure'),
    Input('t-slider', 'value'),
    Input('ph-slider', 'value'),
    Input('input_pka', 'value'),
    Input('input_pkb', 'value'),
    Input('input_t_ref', 'value'),
    Input('input_vmax_ref', 'value'),
    Input('input_km_ref', 'value'),
    Input('input_ea', 'value'),
    Input('input_deltah', 'value'),
    Input('input_t_opt', 'value'),
    Input('input_sigma', 'value'),

)
 #Atualiza Gráfico
def mudar_imagem(t, ph, pka_inp, pkb_inp, t_inp, vmax_inp, km_inp, ea_inp, deltah_inp, t_opt_inp, sigma_inp):

    #Recebe parâmetros
    pKa = float(pka_inp)
    pKb = float(pkb_inp)
    Vmax_ref = float(vmax_inp)
    Ea = float(ea_inp)
    T_ref = float(t_inp)
    Km_ref = float(km_inp)
    deltaH = float(deltah_inp)
    t_opt = float(t_opt_inp)
    sigma = float(sigma_inp)

    #Concentração x Velocidade
    s = np.linspace(0, 20, 500)
    v = v0(s, t, ph, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, t_opt, sigma)

    #Cria Figura
    fig = go.Figure()

    #Adiciona pontos/linhas
    fig.add_trace(go.Scatter(
        x=s,
        y=v,
        mode='lines',
    ))

    #Define visual
    fig.update_layout(
        title="Michaelis",
        xaxis=dict(range=[0, 20]),
        yaxis=dict(range=[0, 50]),
        xaxis_title="Concentração do Substrato (mol/L)",
        yaxis_title="Velocidade da reação (mol/L.s)"
    )

    return fig

#Roda app
if __name__ == '__main__':
    app.run(debug=True, port=8034)   

# Grafico em função de temperatura e pH

In [8]:
dados_enzimas = {
    "Glicólise": {
        "Hexoquinase": {
            "Vmax": 10,
            "Km": 0.05,
            "pKa": 7.0,
            "pKb": 9.0,
            "Delta_H": -30,
            "Ea": 50,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 310,  
            "sigma": 15.0   
        },
        "PFK-1": {
            "Vmax": 20,
            "Km": 0.1,
            "pKa": 7.2,
            "pKb": 9.1,
            "Delta_H": -25,
            "Ea": 45,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 308,
            "sigma": 12.0
        }
    },

    "Ciclo de Krebs": {
        "Citrato sintase": {
            "Vmax": 5,
            "Km": 0.015,
            "pKa": 7.4,
            "pKb": 9.0,
            "Delta_H": -20,
            "Ea": 40,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 310,
            "sigma": 18.0
        },
        "Isocitrato desidrogenase": {
            "Vmax": 8,
            "Km": 0.05,
            "pKa": 7.5,
            "pKb": 8.8,
            "Delta_H": -15,
            "Ea": 42,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 309,
            "sigma": 10.0
        },
        "Alfa-cetoglutarato desidrogenase": {
            "Vmax": 7,
            "Km": 0.02,
            "pKa": 7.3,
            "pKb": 9.0,
            "Delta_H": -18,
            "Ea": 43,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 311,
            "sigma": 14.0
        }
    },

    "Via das Pentoses Fosfato": {
        "Glicose-6-fosfato desidrogenase": {
            "Vmax": 12,
            "Km": 0.04,
            "pKa": 7.1,
            "pKb": 8.9,
            "Delta_H": -22,
            "Ea": 48,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 310,
            "sigma": 16.0
        }
    },

    "Beta-Oxidação": {
        "Acil-CoA desidrogenase": {
            "Vmax": 6,
            "Km": 0.03,
            "pKa": 7.3,
            "pKb": 8.7,
            "Delta_H": -28,
            "Ea": 50,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 312,
            "sigma": 20.0
        }
    },

    "Síntese de Colesterol": {
        "HMG-CoA redutase": {
            "Vmax": 4,
            "Km": 0.01,
            "pKa": 7.2,
            "pKb": 9.2,
            "Delta_H": -17,
            "Ea": 55,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 307,
            "sigma": 8.0
        }
    },

    "Ciclo da Ureia": {
        "Carbamoil-fosfato sintetase I": {
            "Vmax": 3,
            "Km": 0.02,
            "pKa": 7.6,
            "pKb": 9.0,
            "Delta_H": -24,
            "Ea": 47,
            "R": 8.314,
            "T_ref": 298,
            "T_opt": 313,
            "sigma": 13.0
        }
    }
}

In [ ]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import math

#Função da Velocidade
def v0(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):
    term1 = 1 / (1 + 10**(pKa - pH) + 10**(pH - pKb))
    exp1 = math.exp((-Ea*1000 / 8.314) * (1/(T+273) - 1/(T_ref)))
    exp2 = math.exp((deltaH*1000 / 8.314) * (1/(T+273) - 1/(T_ref)))
    exp3 = math.exp(-((T+273 - T_opt) / sigma)**2)
    
    
    numerador = Vmax_ref * exp1 * S
    denominador = Km_ref * exp2 + S
    
    return term1 * exp3 * (numerador / denominador)

#Cria figura (provisória)
fig = go.Figure()

#Cria app
app = Dash(__name__)

#Define visual do app
app.layout = html.Div([
    #Cria Dropdown da Rota Metabólica
    html.Label("Rota Metabólica", style={'color': 'black'}),
    dcc.Dropdown(
        id='dropdown-rota',
        options=[{'label': rota, 'value': rota} for rota in dados_enzimas],
        value='Glicólise'
    ),

    #Cria Dropdown da Enzima
    html.Label("Enzima", style={'color': 'black'}),
    dcc.Dropdown(
        id='dropdown-enzima',
    ),

    #Cria Slider da Temperatura
    html.Label("Temperatura:", style={'color': 'black'}),
    dcc.Slider(
        id='t-slider',
        min=20,
        max=70,
        step=1,
        value=25
    ),

    html.Br(),

    #Cria Slider do pH
    html.Label("pH:", style={'color': 'black'}),
    dcc.Slider(
        id='ph-slider',
        min=5.5,
        max=9.5,
        step=0.1,
        value=7
    ),

    #Insere gráfico
    dcc.Graph(id='Velocidade(concentração)')
])

#Define Inputs e Outputs (Dropdown Rota Metabólica)
@app.callback(
    Output('dropdown-enzima', 'options'),
    Input('dropdown-rota', 'value')
)
#Atualiza Enzimas
def atualizar_opcoes_enzimas(rota):
    enzimas = dados_enzimas.get(rota, {})
    return [{'label': enzima, 'value': enzima} for enzima in enzimas]

#Define Inputs e Outputs (Dropdown Enzimas)
@app.callback(
    Output('dropdown-enzima', 'value'),
    Input('dropdown-enzima', 'options')
)
#Seleciona primeira enzima disponível
def definir_enzima_padrao(opcoes):
    if opcoes:
        return opcoes[0]['value']
    return None

#Define Inputs e Outputs (Gráfico)
@app.callback(
    Output('Velocidade(concentração)', 'figure'),
    Input('t-slider', 'value'),
    Input('ph-slider', 'value'),
    Input('dropdown-rota', 'value'),
    Input('dropdown-enzima', 'value'),
)
#Atualiza Gráfico
def mudar_imagem(t, ph, rota, enzima):

    if None in [rota, enzima]:
        return go.Figure()
    
    #Recebe parâmetros
    pKa = dados_enzimas[rota][enzima]['pKa']
    pKb = dados_enzimas[rota][enzima]['pKb']
    Vmax_ref = dados_enzimas[rota][enzima]['Vmax']
    Ea = dados_enzimas[rota][enzima]['Ea']
    T_ref = dados_enzimas[rota][enzima]['T_ref']
    Km_ref = dados_enzimas[rota][enzima]['Km']
    deltaH = dados_enzimas[rota][enzima]['Delta_H']
    t_opt = dados_enzimas[rota][enzima]['T_opt']
    sigma = dados_enzimas[rota][enzima]['sigma']

    #Concentração x Velocidade
    s = np.linspace(0, 20, 500)
    v = v0(s, t, ph, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, t_opt, sigma)

    #Cria figura
    fig = go.Figure()

    #Adiciona pontos/linhas
    fig.add_trace(go.Scatter(
        x=s,
        y=v,
        mode='lines',
    ))

    #Define visual
    fig.update_layout(
        title=f"Velocidade da reação -> {rota} - {enzima}",
        xaxis=dict(range=[0, 5]),
        yaxis=dict(range=[0, 10]),
        xaxis_title="Concentração do Substrato (mol/L)",
        yaxis_title="Velocidade da reação (mol/L.s)"
    )

    return fig

#Roda app
if __name__ == '__main__':
    app.run(debug=True, port=8039)   

# Gráfico 3D Temperatura e pH

In [ ]:
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import numpy as np
import math

# Função de velocidade
def velocidade_da_reacao(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):
    T_K = T + 273.15
    T_ref_K = T_ref + 273.15
    T_opt_K = T_opt + 273.15

    term1 = 1 / (1 + 10**(pKa - pH) + 10**(pH - pKb))
    exp1 = math.exp((-Ea * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    exp2 = math.exp((deltaH * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    fator_f = math.exp(-((T_K - T_opt_K) / sigma)**2)

    numerador = Vmax_ref * exp1 * fator_f * S
    denominador = Km_ref * exp2 + S

    return term1 * (numerador / denominador)

#Cria app
app = Dash(__name__)

#Define estilo dos elementos
label_style = {
    'color': '#333333',
    'fontWeight': 'bold',
    'fontSize': '14px',
    'marginBottom': '6px',
    'display': 'block'
}

input_style = {
    'width': '100%',
    'padding': '8px 12px',
    'border': '1px solid #ccc',
    'borderRadius': '6px',
    'fontSize': '14px',
    'fontWeight': '400',
    'outline': 'none',
    'boxShadow': 'none',
    'transition': 'border-color 0.3s ease'
}

container_style = {
    'maxWidth': '1100px',
    'margin': '40px auto',
    'padding': '30px',
    'backgroundColor': '#f5f5f7',
    'borderRadius': '12px',
    'fontFamily': 'Trebuchet MS, sans-serif',
    'color': '#1d1d1f',
    'boxShadow': '0 4px 12px rgba(0,0,0,0.05)'
}

title_style = {
    'textAlign': 'center',
    'color': '#1d1d1f',
    'marginBottom': '40px',
    'fontWeight': '600',
    'fontSize': '28px'
}

#Define visual do app
app.layout = html.Div([
    html.H1("Simulação - Cinética Enzimática", style=title_style),

    html.Div([
        html.Div([
            html.Div([
                html.Label("Concentração do Substrato (mol/L)", style=label_style),
                dcc.Input(id='input_s', type='number', value=10, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Dissociação do Grupo Ácido (pKa)", style=label_style),
                dcc.Input(id='input_pka', type='number', value=6.5, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Dissociação do Grupo Básico (pKb)", style=label_style),
                dcc.Input(id='input_pkb', type='number', value=8, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Velocidade Máxima da Reação (mol/L.s)", style=label_style),
                dcc.Input(id='input_vmax_ref', type='number', value=32, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Energia de Ativação da Reação (kJ/mol)", style=label_style),
                dcc.Input(id='input_ea', type='number', value=50, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Temperatura de Referência (°C)", style=label_style),
                dcc.Input(id='input_tref', type='number', value=25, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Michaelis-Menten (mol/L)", style=label_style),
                dcc.Input(id='input_km_ref', type='number', value=0.4, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Variação de Entalpia (kJ/mol)", style=label_style),
                dcc.Input(id='input_deltah', type='number', value=-20, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Temperatura Ótima em Condições Ideais (°C)", style=label_style),
                dcc.Input(id='input_topt', type='number', value=37, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Sensibilidade Térmica (°C)", style=label_style),
                dcc.Input(id='input_sigma', type='number', value=10, style=input_style)
            ], style={'marginBottom': '10px'}),
        ], style={
            'flex': '0 0 380px',
            'paddingRight': '30px',
            'overflowY': 'auto',
            'maxHeight': '750px',
            'boxSizing': 'border-box'
        }),

        html.Div([
            dcc.Graph(id='graph-3d', style={'height': '700px'})
        ], style={'flex': '1'})
    ], style={'display': 'flex', 'flexDirection': 'row'})
], style=container_style)

#Define Inputs e Outputs
@app.callback(
    Output('graph-3d', 'figure'),
    Input('input_s', 'value'),
    Input('input_pka', 'value'),
    Input('input_pkb', 'value'),
    Input('input_vmax_ref', 'value'),
    Input('input_ea', 'value'),
    Input('input_tref', 'value'),
    Input('input_km_ref', 'value'),
    Input('input_deltah', 'value'),
    Input('input_topt', 'value'),
    Input('input_sigma', 'value')
)

#Atualiza gráfico
def update_graph_3d(S, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):

    #Define Valores
    T_vals = np.linspace(10, 50, 40)
    pH_vals = np.linspace(4, 10, 40)
    T_grid, pH_grid = np.meshgrid(T_vals, pH_vals)
    v_grid = np.vectorize(lambda T, pH: velocidade_da_reacao(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma))(T_grid, pH_grid)

    #Cria gráfico
    fig = go.Figure(data=[go.Surface(
        z=v_grid,
        x=T_vals,
        y=pH_vals,
        colorscale='Viridis',
        colorbar=dict(title='Velocidade (mol/L·s)')
    )])

    #Define visuais do gráficos
    fig.update_layout(
        title="Velocidade da Reação Enzimática",
        title_x=0.5,
        title_font=dict(
            size=20,
            color='#1d1d1f',
            family='Arial, sans-serif',
            weight='bold'
        ),
        scene=dict(
            xaxis_title='Temperatura (°C)',
            yaxis_title='pH',
            zaxis_title='Velocidade (mol/L·s)',
            xaxis=dict(showgrid=True, gridcolor='#e1e1e1', zerolinecolor='#e1e1e1', linecolor='#ccc'),
            yaxis=dict(showgrid=True, gridcolor='#e1e1e1', zerolinecolor='#e1e1e1', linecolor='#ccc'),
            zaxis=dict(showgrid=True, gridcolor='#e1e1e1', zerolinecolor='#e1e1e1', linecolor='#ccc'),
            xaxis_range=[10, 50],
            yaxis_range=[4, 10],
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(color='#1d1d1f')
    )

    return fig

#Roda app
if __name__ == '__main__':
    app.run(debug=True, port=8080)

# Concentração em função do tempo

In [5]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

#Função da Velocidade
def michaelis (conc_subst, km, vmax):
    v_reaction = (vmax*conc_subst)/(km+conc_subst)
    return v_reaction

#Função da Concentração
def concentracao(t, km, vmax, s_i):
    s_f = s_i
    dt = 0.00001
    tempo = 0
    while tempo < t:
        s_f = s_f - michaelis(s_f, km, vmax)*(dt)
        tempo += dt
    return max(s_f, 0)

#Cria Figura (provisória)
fig = go.Figure()

#Cria app
app = Dash(__name__)

#Define visual do app
app.layout = html.Div([
    #Cria Slider da Concentração Inicial
    html.Label("s0:", style={'color': 'white'}),
    dcc.Slider(
        id='s0-slider',
        min=0.1,
        max=2.0,
        step=0.1,
        value=1
    ),

    #Cria Slider do Km
    html.Label("Km:", style={'color': 'white'}),
    dcc.Slider(
        id='km-slider',
        min=0.1,
        max=2.0,
        step=0.1,
        value=0.4
    ),

    html.Br(),

    #Cria Slider do Vmáx
    html.Label("Vmax:", style={'color': 'white'}),
    dcc.Slider(
        id='vmax-slider',
        min=10,
        max=50,
        step=1,
        value=32
    ),

    #Insere Gráfico
    dcc.Graph(id='Velocidade(concentração)', figure=fig)
])

#Define Inputs e Outputs
@app.callback(
    Output('Velocidade(concentração)', 'figure'),
    Input('s0-slider', 'value'),
    Input('km-slider', 'value'),
    Input('vmax-slider', 'value')
)

#Altera Gráfico
def mudar_imagem(s0, km, vmax):

    #Tempo x Concentração
    t = np.linspace(0, 0.2, 500)
    s = np.array([])
    for tempo in t:
        s = np.append(s, [concentracao(tempo, km, vmax, s0)])

    #Cria figura
    fig = go.Figure()

    #Adiciona pontos/linhas
    fig.add_trace(go.Scatter(
        x=t,
        y=s,
        mode='lines',
    ))

    #Define visual do gráfico
    fig.update_layout(
        title="Concentração em função do tempo",
        xaxis=dict(range=[0, 0.2]),
        yaxis=dict(range=[0, 2]),
        xaxis_title="Concentração do Substrato (mol/L)",
        yaxis_title="Velocidade da reação (mol/L.s)"
    )


    return fig

#Roda app
if __name__ == '__main__':
    app.run(debug=True, port=8038)   

# Concentração em função do tempo - Reação em cadeia

In [ ]:
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

#Função de Velocidade
def michaelis (conc_subst, km, vmax):
    v_reaction = (vmax*conc_subst)/(km+conc_subst)
    return v_reaction

#Função de Concentração
def concentracao(t, km, vmax, s_i):
    s_f = s_i
    dt = 0.00001
    tempo = 0
    while tempo < t:
        s_f = s_f - michaelis(s_f, km, vmax)*(dt)
        tempo += dt
    return max(s_f, 0)

#Parâmetros
s0 = 1

#Cria figura (provisória)
fig = go.Figure()

#Cria app
app = Dash(__name__)

#Define visual do app
app.layout = html.Div([
    html.Div([
        html.Div([
            # Inputs grupo 1
            html.Div([
                #Cria Slider Km
                html.Label("Reação 1 - Km:",
                           style={'color': 'white', 'margin-right': '10px'}),
                dcc.Input(id='km-input1',
                          type='number',
                          value=0.4,
                          style={'width': '80px', 'margin-right': '20px'}),
                #Cria Slider Vmáx
                html.Label("Vmáx:",
                           style={'color': 'white', 'margin-right': '10px'}),
                dcc.Input(id='vmax-input1',
                          type='number',
                          value=32,
                          style={'width': '80px'})
            ], 
            #Alinha Inputs grupo 1
            style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '20px'}),
            
            # Inputs grupo 2
            html.Div([
                #Cria Slider Km
                html.Label("Reação 2 - Km:",
                           style={'color': 'white', 'margin-right': '10px'}),
                dcc.Input(id='km-input2',
                          type='number',
                          value=0.7,
                          style={'width': '80px', 'margin-right': '20px'}),
                #Cria Slider Vmáx
                html.Label("Vmáx:",
                           style={'color': 'white', 'margin-right': '10px'}),
                dcc.Input(id='vmax-input2',
                          type='number',
                          value=26,
                          style={'width': '80px'})
            ],
            #Alinha Inputs grupo 2
            style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '20px'}),
            
            # Inputs grupo 3
            html.Div([
                #Cria Slider Km
                html.Label("Reação 3 - Km:", 
                           style={'color': 'white', 'margin-right': '10px'}
                           ),
                dcc.Input(id='km-input3',
                          type='number',
                          value=0.5,
                          style={'width': '80px', 'margin-right': '20px'}),
                #Cria Slider Vmáx
                html.Label("Vmáx:",
                           style={'color': 'white', 'margin-right': '10px'}),
                dcc.Input(id='vmax-input3',
                          type='number',
                          value=40, 
                          style={'width': '80px'})
            ], 
            #Alinha Inputs grupo 3
            style={'display': 'flex', 'align-items': 'center'})
        ], 
        style={'display': 'flex', 'flex-direction': 'column'}),
        
        # Gráfico
        dcc.Graph(id='Velocidade(concentração)', figure=fig)
    ], style={'padding': '20px'})
])

#Define Inputs e Outputs
@app.callback(
    Output('Velocidade(concentração)', 'figure'),
    Input('km-input1', 'value'),
    Input('vmax-input1', 'value'),
    Input('km-input2', 'value'),
    Input('vmax-input2', 'value'),
    Input('km-input3', 'value'),
    Input('vmax-input3', 'value'),
)

#Altera Gráfico
def mudar_imagem(km1, vmax1, km2, vmax2, km3, vmax3):

    #Tempo x Concentrações (Vazios)
    t = np.linspace(0, 0.5, 250)
    s1 = np.array([])  # Substrato inicial
    s2 = np.array([])  # Produto 1
    s3 = np.array([])  # Produto 2
    s4 = np.array([])  # Produto final
    
    #Concentrações iniciais
    current_s1 = s0
    current_s2 = 0
    current_s3 = 0
    current_s4 = 0
    
    #Passo de tempo
    dt = t[1] - t[0]
    
    for tempo in t:
        #Adiciona concentração inicial
        if tempo == 0:
            s1 = np.append(s1, current_s1)
            s2 = np.append(s2, current_s2)
            s3 = np.append(s3, current_s3)
            s4 = np.append(s4, current_s4)
        #Adiciona demais concentrações
        else:
            #Calcula as taxas de reação
            v1 = michaelis(current_s1, km1, vmax1)
            v2 = michaelis(current_s2, km2, vmax2)
            v3 = michaelis(current_s3, km3, vmax3)
            
            #Calcula variação de concentração
            delta_s1 = -v1 * dt
            delta_s2 = (v1 - v2) * dt
            delta_s3 = (v2 - v3) * dt
            delta_s4 = v3 * dt
            
            #Calcula concentração final
            current_s1 = max(0, current_s1 + delta_s1)
            current_s2 = max(0, current_s2 + delta_s2)
            current_s3 = max(0, current_s3 + delta_s3)
            current_s4 = max(0, current_s4 + delta_s4)
            
            #Adiciona Concentração
            s1 = np.append(s1, current_s1)
            s2 = np.append(s2, current_s2)
            s3 = np.append(s3, current_s3)
            s4 = np.append(s4, current_s4)
     
    #Cria figura 
    fig = go.Figure()

    #Adiciona pontos/linhas substância 1
    fig.add_trace(go.Scatter(
        x=t,
        y=s1,
        mode='lines',
        name= 'composto 1',
    ))
    #Adiciona pontos/linhas substância 2
    fig.add_trace(go.Scatter(
        x=t,
        y=s2,
        mode='lines',
        name= 'composto 2',
    ))
    #Adiciona pontos/linhas substância 3
    fig.add_trace(go.Scatter(
        x=t,
        y=s3,
        mode='lines',
        name= 'composto 3',
    ))
    #Adiciona pontos/linhas substância 4
    fig.add_trace(go.Scatter(
        x=t,
        y=s4,
        mode='lines',
        name= 'composto 4',
    ))

    #Define visual
    fig.update_layout(
        title="Concentração de Substrato em Cadeia (Michaelis-Menten)",
        xaxis_title="Tempo (s)",
        yaxis_title="Concentração do Substrato (mol/L)",
        yaxis_range=[0, s0],
        xaxis_range=[0, 0.2]
    )
    return fig

#Roda app
if __name__ == '__main__':
    app.run(debug=True, port=8035)
